In [ ]:
import os
import re

import numpy as np

import biobeam
import gputools
import pyopencl
from matplotlib import pyplot as plt
from skimage.io import imsave
from skimage.io import imread_collection


%matplotlib inline
plt.rcParams['figure.figsize'] = [12,12]


In [ ]:
def split_str_digit(s):
    """
    split s into numeric (integer) and non-numeric parts
    return split as a tuple of ints and strings
    """
    res = []
    for m in re.finditer('(\d*)(\D*)', s):
        for g in m.groups():
            if g != '':
                try:
                    res.append(int(g))
                except ValueError:
                    res.append(g)
    return tuple(res)


def sim_lightsheet(base_path, out_dir, phantom_dir='sim-phantom', in_pattern='sim(\d+).tif', right_illum=False,
                   physical_dims=(800,800,100), ls_pos=400, lam=500, out_pattern='sim-biobeam{}.tif'):
    
    in_path = os.path.join(base_path, phantom_dir)
    files = os.listdir(in_path)
    files.sort(key=lambda f: split_str_digit(f))

    # read raw phantom
    ic = imread_collection([os.path.join(in_path, f) for f in files])
    img = ic.concatenate()

    # zero-pad image for conv
    img = np.pad(img, ((16,16),(0,0),(0,0)), "constant")
    
    # simulate right illumination with flip of x-axis
    if right_illum:
        img = np.flip(img, 2)
    
    # setup signal and refractive index
    signal = img
    dn = (np.sqrt(signal / np.max(signal))) * 0.01

    #create a microscope simulator
    m = biobeam.SimLSM_Cylindrical(dn = dn, signal = signal, zfoc_illum=physical_dims[0]/2 - ls_pos,
                       NA_illum= .1, NA_detect=.45,
                       n_volumes=2, lam=lam/1000,
                       size = physical_dims, n0 = 1.33)
    
    
    # make outdir if necessary
    out_path = os.path.join(base_path, out_dir)
    if not os.path.exists(out_path):
        os.makedirs(out_path)

    out = np.zeros((img.shape[0] - 16*2,) + img.shape[1:], dtype=img.dtype)
    for i in range(out.shape[0]):

        cz = i - (img.shape[0] - 16*2) // 2
        cz = cz * m._bpm_detect.units[-1]

        image = m.simulate_image_z(cz=cz, psf_grid_dim=(16,16), conv_sub_blocks=(4,4))
        out[i] = image[16] if not right_illum else np.flip(image[16], 2)

        # save immediately
        imsave(os.path.join(out_path, out_pattern.format(i)), out[i])
        print("{} of {} done".format(i+1, out.shape[0]))
    

In [ ]:
if __name__ == '__main__':
    
    base_dir = '/scratch/hoerl/sim_tissue/'
    
    params = [
        (False, 250, 400),
        (False, 400, 400),
        (False, 550, 400),
        (True, 250, 400),
        (True, 400, 400),
        (True, 550, 400),
        (False, 250, 600),
        (False, 400, 600),
        (False, 550, 600),
        (True, 250, 600),
        (True, 400, 600),
        (True, 550, 600)
    ]
    
    for (right_illum, ls_pos, lam) in params:
        out_dir = 'sim-biobeam-illum_{}-ls_pos_{}-lam-{}'.format('right' if right_illum else 'left', ls_pos, lam)
        #sim_lightsheet(base_dir, out_dir, right_illum=right_illum, ls_pos=ls_pos, lam=lam)
    print('ALL DONE.')
    

In [ ]:
# illumination from right side
right_illum = True
# phantom size in microns
physical_dims = (400,400,50)
# LS position
ls_pos = 200

In [ ]:
# load tiff series test data

sequence_pattern = 'sim(\d+).tif'
in_path = '/scratch/hoerl/sim_tissue/sim-phantom/'

files = os.listdir(in_path)
files.sort(key=lambda f: split_str_digit(f))
files

ic = imread_collection([os.path.join(in_path, f) for f in files])
img = ic.concatenate()

img = np.pad(img, ((16,16),(0,0),(0,0)), "constant")
if right_illum:
    img = np.flip(img, 2)

In [ ]:
signal = img
dn = (signal / np.max(signal)) * 0.01

#create a microscope simulator
m = biobeam.SimLSM_Cylindrical(dn = dn, signal = signal, zfoc_illum=physical_dims[0]/2 - ls_pos,
                       NA_illum= .1, NA_detect=.45,
                       n_volumes=2,
                       size = physical_dims, n0 = 1.33)

In [ ]:
out_fmt = 'sim-bbeam-{}-{}.tif'
out_path = '/scratch/hoerl/sim_tissue/sim-bbeam-right/'

if not os.path.exists(out_path):
    os.makedirs(out_path)

out = np.zeros((img.shape[0] - 16*2,) + img.shape[1:], dtype=img.dtype)

for i in range(out.shape[0]):
    
    cz = i - (img.shape[0] - 16*2) // 2
    cz = cz * m._bpm_detect.units[-1]

    image = m.simulate_image_z(cz=cz, psf_grid_dim=(16,16), conv_sub_blocks=(4,4))
    out[i] = image[16] if not right_illum else np.flip(image[16], 2)
    
    # save immediately
    imsave(os.path.join(out_path, out_fmt.format("left" if not right_illum else "right", i)), out[i])
    
    print("{} of {} done".format(i+1, out.shape[0]))

In [ ]:
_ = """

pl = pyopencl.get_platforms()[0]
pl.get_devices()
os.environ['PYOPENCL_COMPILER_OUTPUT'] = '1'

gputools.init_device(id_platform = 0, id_device = 1)

signal = read_tiff_stack('/Users/david/Desktop/Image 0-1.tif', False)
signal = np.swapaxes(signal, 0,2)
dn = (signal > 0) * 0.01

#create a microscope simulator
m = biobeam.SimLSM_Cylindrical(dn = dn, signal = signal,
                       NA_illum= .1, NA_detect=.45,
                       n_volumes=1,
                       size = (128,512,512), n0 = 1.33)

image = m.simulate_image_z(cz=0, psf_grid_dim=(16,16), conv_sub_blocks=(2,2))


print(image.shape)
plt.imshow(image[16])
"""